In [14]:
!git clone https://github.com/jackrubin1/econ8310-assignment1.git
%cd econ8310-assignment1

Cloning into 'econ8310-assignment1'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 95 (delta 36), reused 27 (delta 27), pack-reused 50 (from 1)
Receiving objects: 100% (95/95), 100.14 KiB | 3.58 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1


In [17]:
import os
import pandas as pd
import numpy as np
from prophet import Prophet

train_path = "assignment_data_train.csv"
train_url = "https://raw.githubusercontent.com/jackrubin1/econ8310-assignment1/main/assignment_data_train.csv"

if os.path.exists(train_path):
    train_data = pd.read_csv(train_path)
else:
    train_data = pd.read_csv(train_url)

train_data["Timestamp"] = pd.to_datetime(train_data["Timestamp"])
train_data = train_data.sort_values("Timestamp")

prophet_train = train_data.rename(columns={"Timestamp": "ds", "trips": "y"})[["ds", "y"]]

m = Prophet(
    yearly_seasonality=False,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode="additive"
)

m.add_seasonality(name="hourly", period=24, fourier_order=10)

m.fit(prophet_train)

future = m.make_future_dataframe(periods=744, freq="h")
forecast = m.predict(future)

pred = forecast["yhat"].tail(744).to_numpy()
pred = np.maximum(pred, 0)
